# Miniproyecto 3

### David Soto 17551 & Miguel Valle 17102

In [1]:
### Librerias a utilizar para la simulacion
import numpy as np
import random

### Función generadora de tiempos de estadía/salida

In [2]:
# Funcion para calcular el proximo tiempo de llegada
def tiempoProximaSalida(t, lamda_max, homo):
    if homo:
        return t - ((1/lamda_max)*np.log(random.random()))
    else:
        while True:
            u = random.random()
            t = t - ((1/lamda_max)*np.log(u))
            u = random.random()
            if u <= np.random.poisson(t)/lamda_max:
                return t

### Funcion simuladora de eventos

In [3]:
### Funcion que determina cómo procede el evento segun el tiempo dela simulacion
def evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed):
    ### Si la proxima peticion es el ultimo registrado en el sistema, entonces hay un servidor libre que lo puede atender
    if ta == min(ta,min(td)): 
        t = ta
        Na = Na + 1
        ta = tiempoProximaSalida(t, lamda_max, True)
        A[Na] = t
        ### Si la cantidad de peticiones en el sistema es menor a la cantidad de cpus
        ### es porque la proxima peticion puede ser procesada por un servidor
        if n < no_cpus:
            for i in range(no_cpus):
                if ids[i] == 0:
                    ids[i] = Na
                    NT = np.append(NT, t - A[Na])
                    td[i] = t + np.random.exponential(speed)
                    busyTime[i] = busyTime[i] + td[i] - t
                    break
        n = n + 1
    else:
        ### Esto ocurre en caso de que no haya un servidor desocupado al momento de llegar
        ti = np.argmin(td)
        t = td[ti]
        C[ti] = C[ti] + 1
        D[ids[ti]] = t
        ### En caso la cantidad de clientes sea menor o igual a la cantidad de servidores
        ### Se libera un CPU
        if n <= no_cpus:
            ### Se elimina a la ultima peticion del ID register
            ids[ti] = 0
            td[ti] = np.infty
        ### Si todavia hay mas clientes por atender
        else: 
            indice = max(ids) + 1
            ### Se actualiza el indice
            ids[ti] = indice
            NT = np.append(NT , t - A[indice])
            td[ti] = t + np.random.exponential(speed)
            busyTime[ti] = busyTime[ti] + td[ti] - t
        n = n - 1
        
    ### Retornamos las variables para actualizacion y seguir el proceso de la simulacion 
    return t, n, Na, C, busyTime, A, D, NT, ta, td, ids

### Funcion simuladora del procesamiento de peticiones

In [4]:
### Funcion para realizar la simulacion
def simular(duracion_simulacion, lamda_max, speed,no_cpus):
    ### Variable de tiempo de simulacion
    t = 0
    
    # Variables contadores
    Na = 0
    C = np.zeros(no_cpus) 
    busyTime = np.zeros(no_cpus)
    
    ### Variable estado del sistema
    n = 0

    ### Variables para el control de datos
    A = {}
    D = {}
    NT = []
    
    # Variables de eventos de tiempos de llegada y salida
    ta = tiempoProximaSalida(t,lamda_max, True)
    td = np.zeros(no_cpus) + np.infty
    ids = np.zeros(no_cpus)
    
    # Se realiza la simulacion hasta que se llegue al tiempo final de esta
    while t < duracion_simulacion:
        ### Realizamos la simulacion de un evento al timepo t
        t, n, Na, C, busyTime, A, D, NT, ta, td, ids = evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed)
        
    ### Obtenemos el tiempo de salida de la ultima solicitud atendida
    copiaArreglo = td
    tiemposSalida = []
    for i in copiaArreglo:
        if not np.isinf(i):
            tiemposSalida.append(i)

    ### Mostramos los resultados de las variables de la somulacion
    print("Solicitudes atendidas: ", C)
    print("Tiempos de ocupacion de servidores: ", busyTime)
    print("Tiempos libres de servidores: ", np.maximum(np.ones(no_cpus)*duracion_simulacion - busyTime,0))
    print("Tiempo de solicitudes en cola: ", np.round(sum(NT),5))
    print("Promedio de solicitudes en cola:", np.round(np.mean(NT),5))
    print("Tiempo de salida de la ultima solicitud atendida: ", np.round(max(tiemposSalida),8))

## Task 1 - Simulacion con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto -  Gorilla Megacomputing

In [5]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [143382.]
Tiempos de ocupacion de servidores:  [23.93425487]
Tiempos libres de servidores:  [36.06574513]
Tiempo de solicitudes en cola:  15.55682
Promedio de solicitudes en cola: 0.00011
Tiempo de salida de la ultima solicitud atendida:  60.00126561


### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [6]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [28764. 26913. 23885. 20198. 16158. 11822.  7951.  4599.  2397.  1199.]
Tiempos de ocupacion de servidores:  [48.11684241 44.4509369  39.56258611 33.61179999 26.84056995 19.63608776
 13.11151263  7.71126034  4.07053196  2.00881888]
Tiempos libres de servidores:  [11.88315759 15.5490631  20.43741389 26.38820001 33.15943005 40.36391224
 46.88848737 52.28873966 55.92946804 57.99118112]
Tiempo de solicitudes en cola:  0.23868
Promedio de solicitudes en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  60.00172749


## Task 2 - Determinacion de numero de servidores para no tener cola con Ants con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [7]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 15

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [2.9063e+04 2.6512e+04 2.3658e+04 2.0074e+04 1.5879e+04 1.1699e+04
 7.6630e+03 4.5490e+03 2.3700e+03 1.1340e+03 4.6000e+02 1.8000e+02
 6.2000e+01 2.4000e+01 1.1000e+01]
Tiempos de ocupacion de servidores:  [4.78375281e+01 4.41987114e+01 3.94448573e+01 3.33282141e+01
 2.65220256e+01 1.94369722e+01 1.25761346e+01 7.50383279e+00
 3.92878069e+00 1.80403428e+00 7.70793091e-01 2.80573658e-01
 1.26497622e-01 3.40182364e-02 2.27523869e-02]
Tiempos libres de servidores:  [12.16247194 15.80128856 20.55514272 26.67178594 33.47797444 40.56302784
 47.42386538 52.49616721 56.07121931 58.19596572 59.22920691 59.71942634
 59.87350238 59.96598176 59.97724761]
Tiempo de solicitudes en cola:  0.00117
Promedio de solicitudes en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  60.00213755


## Task 3 - Simulacion con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Gorilla Megacomputing

In [8]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [359607.]
Tiempos de ocupacion de servidores:  [59.95529386]
Tiempos libres de servidores:  [0.04470614]
Tiempo de solicitudes en cola:  45629.41903
Promedio de solicitudes en cola: 0.12689
Tiempo de salida de la ultima solicitud atendida:  60.00011025


### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [9]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [36138. 36274. 35776. 35830. 36065. 35942. 35626. 35637. 35802. 35627.]
Tiempos de ocupacion de servidores:  [59.86085379 59.83483114 59.81928931 59.78533926 59.72881215 59.71058329
 59.61523869 59.54174251 59.45295013 59.34578218]
Tiempos libres de servidores:  [0.13914621 0.16516886 0.18071069 0.21466074 0.27118785 0.28941671
 0.38476131 0.45825749 0.54704987 0.65421782]
Tiempo de solicitudes en cola:  10709.4088
Promedio de solicitudes en cola: 0.02985
Tiempo de salida de la ultima solicitud atendida:  60.0049889


## Task 4 - Determinacion de numero de servidores para no tener cola con Ants con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [10]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 30

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [3.2482e+04 3.1890e+04 3.1397e+04 3.0556e+04 3.0005e+04 2.8548e+04
 2.6864e+04 2.5770e+04 2.3450e+04 2.1242e+04 1.8639e+04 1.5721e+04
 1.2786e+04 1.0032e+04 7.4030e+03 5.1810e+03 3.4800e+03 2.2130e+03
 1.2420e+03 6.8400e+02 3.8300e+02 1.6400e+02 7.5000e+01 3.0000e+01
 1.2000e+01 5.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
Tiempos de ocupacion de servidores:  [5.45789299e+01 5.37055653e+01 5.26096126e+01 5.14012904e+01
 4.95870058e+01 4.78225224e+01 4.54815657e+01 4.24228220e+01
 3.90356110e+01 3.53883388e+01 3.10172792e+01 2.62944013e+01
 2.16894282e+01 1.66298692e+01 1.24049054e+01 8.71047347e+00
 5.80287701e+00 3.58920438e+00 2.04902650e+00 1.11868579e+00
 6.19608109e-01 2.93321572e-01 1.39986225e-01 5.80918175e-02
 1.89184039e-02 1.00958011e-02 4.07435031e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Tiempos libres de servidores:  [ 5.42107008  6.29443468

## Task 5 - Recomendacion a la junta directiva

Dado que se observa una mejora clara en el tiempo de espera para las solicitudes hechas, se recomienda alquilar 15 servidores para un numero de solicitudes de 2400, y de alquilar 30 servidores para 6000 solicitudes, ya que aunque alquilar más servidores podría reducir aún más el tiempo de espera, no se estaran utilizando todos cuando no se tenga una cantidad de solicitudes muy elevada, por lo cual sería un desperdicio del presupuesto. Es preferible tener menos servidores, pero que todos se estén aprovechando durante un flujo normal de solicitudes realizada, tomando como línea base 2400 y límite superior valores cercanos a 6000. Por lo tanto la cantidad de servidores que se recomienda alquilar se encuentra en un rango entre 15 y 30 servidores.